In [1]:
# Lucas-Lehmer (LL) test for Mersenne primes

## imports
import numpy as np



23


In [12]:
## function definitions

# get the 'p'th mersenne prime
def M(p):
    return 2 ** p - 1

# tell whether a number is prime via test division
def isPrime(n):
    if n == 2: return True
    if n % 2 == 0 or n < 2: return False
    RSp = int(n ** 0.5) + 1
    for i in range(3, RSp+1, 2):
        if n % i == 0:
            return False
    return True


# perform a lucas lehmer test on 2**p-1, return whether it was prime
def LLtest(p):
    # hard test
    if p == 2: return True
    
    # perform trial division on the exponent
    if not isPrime(p): return False

    
    # calculate the mersenne prime
    Mp = M(p)
    
    # current residue
    Li = 4
    
    # perform tests
    for i in range(p-2):
        Li = pow(Li, 2, Mp) - 2
        
    return (Li % Mp) == 0


# assert 
for i in range(2, 40):
    assert LLtest(i) == isPrime(M(i))

print ("Success!")


Success!


In [33]:


def intMM(a, b):
    
    base = 2 ** 32
    
    # turn into an array with a given base
    def getX(num):
        X = []
        d = num
        
        # while there are more digits
        while d > 0:
            # although this has a higher complexity, it is still be calculated fairly
            #   fast because it can be performed with a bitshift of a fixed number
            #   of bits, so probably O(n)
            d, m = divmod(d, base)
            X.append(m)
        
        return X
    
    # get back an integer from the base
    def fromX(X):
        num = 0
        bi = 1
        
        for x in X:
            # we use multiplication by a base here; this is O(1) in C though;
            #  because it is a fixed size
            num += bi * x
            bi *= base
        
        return int(num)
        
    # arrays
    aX = getX(a)
    bX = getX(b)
    
    # add 0s and pad it
    if len(aX) > len(bX):
        aX.append(0)
        bX += [0] * (len(aX) - len(bX) + 1)
    elif len(aX) < len(bX):
        bX.append(0)
        aX += [0] * (len(bX) - len(aX) + 1)
    else:
        aX.append(0)
        bX.append(0)
    
    # frequencies of digits
    faX = np.fft.rfft(aX)
    fbX = np.fft.rfft(bX)
    
    # pointwise multiplication of frequencies, which convolves the integers
    # think about it:
    #
    #   34
    # x 12
    # ----
    #   68
    #  340
    # ----
    #  408
    #
    # Each of the rows in the second sum is the original sequence (34)
    #   offset by the index of the digit in the second sequence (12),
    #   and scaled by the magnitude
    # This is:
    # sum(sum(A[j] * B[i] * 10**i for j in range(n)) for i in range(n))
    #
    # And so, any particular digit (digit 'd') of the output is:
    # C[d] = sum(A[i - d] * B[i + d] for i in range(n))
    # (not accounting for carry)
    #
    # So, this is a linear sequence that can be modeled as an IR which
    #   the 2 sequences are convolving in
    #
    # Therefore, their product is the convolution of the digits (and carry must be
    #   executed)
    #
    # Since convolve(A, B) == iFFT(FFT(A) .* FFT(B))
    # We can compute it quicker via O(n * logn) FFT algorithm + O(n) multiplication algorithm
    #
    fcX = faX * fbX
    
    # compute inverse FFT
    cX = np.fft.irfft(fcX)
    
    #print (aX, bX, cX)
    
    # return from bases
    return fromX(cX)

print (intMM(1234, 5))


    

6170
